In [37]:
#Initial Imports 
import pandas as pd 
import numpy as np 
import datetime as dt 
import hvplot.pandas
import seaborn as sns
import matplotlib.pyplot as plt
import holoviews as hv
from holoviews import opts
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import plotly.express as px
from pathlib import Path 
%matplotlib inline

# Reading and Cleaning Data

In [38]:
#Reading in files 
coffee_futures = Path('resources/coffee_c_futures.csv')
fx_allcountries = Path('resources/fx_5countries.csv')
irates_brazil = Path('resources/irates_brazil.csv')
irates_colombia = Path('resources/irates_colombia.csv')
irates_ethiopia = Path('resources/irates_ethiopia.csv')
irates_honduras = Path('resources/irates_honduras.csv')
irates_uganda = Path('resources/irates_uganda.csv')
total_prod_csv = Path('resources/TotalProduction.csv')
total_prod = pd.read_csv(total_prod_csv, index_col="Crop year",parse_dates=True).T
exports_csv = Path('resources/Exports.csv')
exports = pd.read_csv(exports_csv, index_col="Crop year").T
opening_stx_csv = Path('resources/OpeningStocks.csv')
opening_stx = pd.read_csv(opening_stx_csv, index_col="Crop years").T
prices_to_growers_csv = Path('resources/PricesToGrowers.csv')
prices_to_growers = pd.read_csv(prices_to_growers_csv, index_col="Calendar years").T
retail_prices_csv = Path('resources/RetailPrices.csv')
retail_prices = pd.read_csv(retail_prices_csv, index_col="Calendar years").T

#Coffee Futures
coffee_futures_data = pd.read_csv(coffee_futures, parse_dates=True, index_col='Date')

#FX 
fx_data = pd.read_csv(fx_allcountries, index_col='Country').T
fx_data = fx_data.iloc[3:]
fx_data = fx_data[['Colombia', 'Honduras', 'Uganda', 'Ethiopia', 'Brazil']]
fx_data.columns = ['Colombia New FX', 'Colombia Old FX', 'Colombia Real FX', 'Honduras New FX', 'Honduras Old FX', 
                   'Honduras Real FX', 'Uganda New FX', 'Uganda Old FX', 'Uganda Real FX', 'Ethiopia New FX', 'Ethiopia Old FX', 
                   'Ethiopia Real FX', 'Brazil New FX', 'Brazil Old FX', 'Brazil Real FX']
fx_data.index.name='Date'


#Brazil Rates
irates_brazil_data = pd.read_csv(irates_brazil, index_col=[0]).T
new_header_brazil = irates_brazil_data.iloc[0]
irates_brazil_data=irates_brazil_data[1:]
irates_brazil_data.columns=new_header_brazil

irates_brazil_data =irates_brazil_data[['Indicator', 'Central Bank Policy Rate', 'Discount Rate', 'Money Market Rate', 
                                        'Treasury Bill Rate', 'Savings Rate', 'Deposit Rate', 'Lending Rate']]
irates_brazil_data.reset_index
irates_brazil_data['Indicator'] = irates_brazil_data['Indicator'].astype(str)
irates_brazil_data.set_index('Indicator', inplace=True)

irates_brazil_data=irates_brazil_data.iloc[3:]

#Colombia Rates
irates_colombia_data = pd.read_csv(irates_colombia, index_col=[0]).T
new_header_colombia = irates_colombia_data.iloc[0]
irates_colombia_data=irates_colombia_data[1:]
irates_colombia_data.columns=new_header_colombia

irates_colombia_data =irates_colombia_data[['Indicator', 'Central Bank Policy Rate', 'Discount Rate', 
                                            'Money Market Rate', 'Deposit Rate', 'Lending Rate']]
irates_colombia_data.reset_index
irates_colombia_data['Indicator'] = irates_colombia_data['Indicator'].astype(str)
irates_colombia_data.set_index('Indicator', inplace=True)

irates_colombia_data=irates_colombia_data.iloc[3:]

#Ethiopia Rates
irates_ethiopia_data = pd.read_csv(irates_ethiopia, index_col=[0]).T
new_header_ethiopia = irates_ethiopia_data.iloc[0]
irates_ethiopia_data=irates_ethiopia_data[1:]
irates_ethiopia_data.columns=new_header_ethiopia

irates_ethiopia_data =irates_ethiopia_data[['Indicator', 'Discount Rate', 'Money Market Rate', 
                                            'Treasury Bill Rate', 'Savings Rate', 'Deposit Rate', 
                                            'Lending Rate', 'Government Bonds']]
irates_ethiopia_data.reset_index
irates_ethiopia_data['Indicator'] = irates_ethiopia_data['Indicator'].astype(str)
irates_ethiopia_data.set_index('Indicator', inplace=True)

irates_ethiopia_data=irates_ethiopia_data.iloc[3:]

#Honduras Rates
irates_honduras_data = pd.read_csv(irates_honduras, index_col=[0]).T
new_header_honduras = irates_honduras_data.iloc[0]
irates_honduras_data=irates_honduras_data[1:]
irates_honduras_data.columns=new_header_honduras

irates_honduras_data =irates_honduras_data[['Indicator', 'Central Bank Policy Rate', 'Savings Rate', 'Savings Rate, Foreign Currency', 
                                            'Deposit Rate', 'Deposit Rate, Foreign Currency', 'Lending Rate', 'Lending Rate, Foreign Currency', 
                                            'Government Bonds']]
irates_honduras_data.reset_index
irates_honduras_data['Indicator'] = irates_honduras_data['Indicator'].astype(str)
irates_honduras_data.set_index('Indicator', inplace=True)

irates_honduras_data=irates_honduras_data.iloc[3:]

#Uganda Rates
irates_uganda_data = pd.read_csv(irates_uganda, index_col=[0]).T
new_header_uganda = irates_uganda_data.iloc[0]
irates_uganda_data=irates_uganda_data[1:]
irates_uganda_data.columns=new_header_uganda

irates_uganda_data =irates_uganda_data[['Indicator', 'Discount Rate', 'Treasury Bill Rate', 'Savings Rate', 'Deposit Rate', 'Lending Rate', 'Government Bonds']]
irates_uganda_data.reset_index
irates_uganda_data['Indicator'] = irates_uganda_data['Indicator'].astype(str)
irates_uganda_data.set_index('Indicator', inplace=True)

irates_uganda_data=irates_uganda_data.iloc[3:]

In [39]:
#Indexes FX
indexes =[]
for index in fx_data.index:
    if '[' in index:
        new_index = index.split('[')[0].strip().split('M')
        new_index = new_index[0] + '/' + new_index[1]
        indexes.append(new_index)
    else:
        indexes.append(index)
        
fx_data.index = pd.Index(indexes)


#Indexes Brazil Rates
indexes =[]
for index in irates_brazil_data.index:
    if 'M' in index:
        new_index = index.split('M')
        new_index = new_index[0] + '/' + new_index[1]
        indexes.append(new_index)
    else:
        indexes.append(index)
        
irates_brazil_data.index = pd.Index(indexes)


#Indexes Colombia Rates
indexes =[]
for index in irates_colombia_data.index:
    if 'M' in index:
        new_index = index.split('M')
        new_index = new_index[0] + '/' + new_index[1]
        indexes.append(new_index)
    else:
        indexes.append(index)
        
irates_colombia_data.index = pd.Index(indexes)

#Indexes Ethiopia Rates
indexes =[]
for index in irates_ethiopia_data.index:
    if 'M' in index:
        new_index = index.split('M')
        new_index = new_index[0] + '/' + new_index[1]
        indexes.append(new_index)
    else:
        indexes.append(index)
        
irates_ethiopia_data.index = pd.Index(indexes)

#Indexes Honduras Rates
indexes =[]
for index in irates_honduras_data.index:
    if 'M' in index:
        new_index = index.split('M')
        new_index = new_index[0] + '/' + new_index[1]
        indexes.append(new_index)
    else:
        indexes.append(index)
        
irates_honduras_data.index = pd.Index(indexes)

#Indexes Uganda Rates
indexes =[]
for index in irates_uganda_data.index:
    if 'M' in index:
        new_index = index.split('M')
        new_index = new_index[0] + '/' + new_index[1]
        indexes.append(new_index)
    else:
        indexes.append(index)
        
irates_uganda_data.index = pd.Index(indexes)


#Drop Nulls

coffee_futures_data = coffee_futures_data[["Settle","Volume"]]
fx_data.dropna(inplace=True)        
irates_brazil_data.dropna(inplace=True)
irates_colombia_data.dropna(inplace=True)
irates_ethiopia_data.dropna(inplace=True)
irates_honduras_data.dropna(inplace=True)
irates_uganda_data.dropna(inplace=True)

#### Further Scrubbing IR Data

In [40]:
for index in irates_brazil_data.index:
    if 'Q' in index:
        irates_brazil_data = irates_brazil_data.drop(index)
for index in irates_brazil_data.index:
    if len(index) <=6:
        irates_brazil_data = irates_brazil_data.drop(index)
        
brazil_ir = irates_brazil_data[["Central Bank Policy Rate"]]
brazil_ir = brazil_ir.rename(columns={"Central Bank Policy Rate": "Brazilian Rates"})
brazil_ir = brazil_ir.replace("...",0)

for index in irates_colombia_data.index:
    if 'Q' in index:
        irates_colombia_data = irates_colombia_data.drop(index)
for index in irates_colombia_data.index:
    if len(index) <=6:
        irates_colombia_data = irates_colombia_data.drop(index)
        
colombia_ir = irates_colombia_data[["Central Bank Policy Rate"]]
colombia_ir = colombia_ir.rename(columns={"Central Bank Policy Rate": "Colombian Rates"})
colombia_ir = colombia_ir.replace("...",0)

for index in irates_ethiopia_data.index:
    if 'Q' in index:
        irates_ethiopia_data = irates_ethiopia_data.drop(index)
for index in irates_ethiopia_data.index:
    if len(index) <=6:
        irates_ethiopia_data = irates_ethiopia_data.drop(index)
        
ethiopia_ir = irates_ethiopia_data[["Discount Rate"]]
ethiopia_ir = ethiopia_ir.rename(columns={"Discount Rate": "Ethiopian Rates"})
ethiopia_ir = ethiopia_ir.replace("...",0)

for index in irates_honduras_data.index:
    if 'Q' in index:
        irates_honduras_data = irates_honduras_data.drop(index)
for index in irates_honduras_data.index:
    if len(index) <=6:
        irates_honduras_data = irates_honduras_data.drop(index)
        
honduras_ir = irates_honduras_data[["Central Bank Policy Rate"]]
honduras_ir = honduras_ir.rename(columns={"Central Bank Policy Rate": "Honduran Rates"})
honduras_ir = honduras_ir.replace("...",0)

for index in irates_uganda_data.index:
    if 'Q' in index:
        irates_uganda_data = irates_uganda_data.drop(index)
for index in irates_uganda_data.index:
    if len(index) <=6:
        irates_uganda_data = irates_uganda_data.drop(index)
        
uganda_ir = irates_uganda_data[["Discount Rate"]]
uganda_ir = uganda_ir.rename(columns={"Discount Rate": "Ugandan Rates"})
uganda_ir = uganda_ir.replace("...",0)

#Cleaning Up ICO Data

total_prod = total_prod[["Brazil", "Colombia", "Honduras", "Uganda","Ethiopia"]]
indexes =[]
for index in total_prod.index:
    if '/' in index:
        new_index = index[0:4]
        indexes.append(new_index)
    else:
        indexes.append(index)
        
total_prod.index = pd.Index(indexes)

exports = exports[["Brazil", "Colombia", "Honduras", "Uganda","Ethiopia"]]
indexes =[]
for index in exports.index:
    if '/' in index:
        new_index = index[0:4]
        indexes.append(new_index)
    else:
        indexes.append(index)
        
exports.index = pd.Index(indexes)

prices_to_growers= prices_to_growers[["Brazil", "Colombia", "Honduras", "Uganda","Ethiopia"]]
indexes =[]
for index in prices_to_growers.index:
    if '/' in index:
        new_index = index[0:4]
        indexes.append(new_index)
    else:
        indexes.append(index)
        
prices_to_growers.index = pd.Index(indexes)
prices_to_growers.columns = ["Brazil", "Brazil2", "Colombia", "Honduras", "Uganda", "Uganda2", "Ethiopia"]
prices_to_growers = prices_to_growers[["Brazil", "Colombia", "Honduras", "Uganda","Ethiopia"]]

opening_stx= opening_stx[["Brazil", "Colombia", "Honduras", "Uganda","Ethiopia"]]
indexes =[]
for index in opening_stx.index:
    if '/' in index:
        new_index = index[0:4]
        indexes.append(new_index)
    else:
        indexes.append(index)
        
opening_stx.index = pd.Index(indexes)

#Setting Index Labels for Combining Data
index_years = ["1990","1991","1992","1993","1994","1995","1996","1997","1998","1999","2000","2001","2002","2003",
             "2004","2005","2006","2007","2008","2009","2010","2011","2012","2013","2014","2015","2016","2017","2018"]

#### Creating ICO Data by Country

In [41]:
# Defining a function to create by country data
def create_country_dataframe(country):
    production = pd.DataFrame()
    production = total_prod[[country]]
    total_exports = pd.DataFrame()
    total_exports = exports[[country]]
    local_prices = pd.DataFrame()
    local_prices = prices_to_growers[[country]]
    stocks = pd.DataFrame()
    stocks = opening_stx[[country]]
    combined_df = pd.DataFrame()
    combined_df = pd.concat([production, total_exports, local_prices, stocks], axis="columns", join="inner")
    combined_df.columns = ["Production", "Exports", "Prices to Growers", "Opening Stocks"] 
    
    return combined_df

In [42]:
#Setting function to specific country variables
brazil_ico = create_country_dataframe("Brazil")
brazil_ico.index = index_years
colombia_ico = create_country_dataframe("Colombia")
colombia_ico.index = index_years
honduras_ico = create_country_dataframe("Honduras")
honduras_ico.index = index_years
uganda_ico = create_country_dataframe("Uganda")
uganda_ico.index = index_years
ethiopia_ico = create_country_dataframe("Ethiopia")
ethiopia_ico.index = index_years

***

***

## Creating Combined Dataframes

In [43]:
# Coffee futures price
kc_monthly = coffee_futures_data.groupby(pd.Grouper(freq='MS')).first()
kc_monthly.index = kc_monthly.index.strftime('%Y/%m')
kc_monthly = kc_monthly["1990/01":"2018/12"]
kc_monthly = kc_monthly[["Settle"]]
kc_monthly.rename(columns={"Settle": "KC Future"}, inplace=True)

# FX Prices
fx_all_monthly = fx_data[["Colombia New FX", "Honduras New FX", "Uganda New FX", "Ethiopia New FX", "Brazil New FX"]]
fx_all_monthly = fx_all_monthly["1990/01":"2018/12"]

#Combining Interest Rates
all_monthly = pd.concat([kc_monthly, fx_all_monthly, brazil_ir, colombia_ir, honduras_ir, uganda_ir], axis="columns", join = "inner")
all_monthly = all_monthly.apply(pd.to_numeric, errors='ignore')
all_monthly.index = pd.to_datetime(all_monthly.index)
all_monthly.index = all_monthly.index.strftime('%Y-%m')
all_monthly = all_monthly.set_index(pd.DatetimeIndex(all_monthly.index))
all_monthly.head()

,KC Future,Colombia New FX,Honduras New FX,Uganda New FX,Ethiopia New FX,Brazil New FX,Brazilian Rates,Colombian Rates,Honduran Rates,Ugandan Rates
1990-01-01,80.55,395.342050,2.031964,373.902940,2.038536,0.000012,0.0,0.0,0.0,55.0
1990-02-01,80.65,405.798743,2.132086,377.808897,2.038536,0.000020,0.0,0.0,0.0,55.0
1990-03-01,96.75,416.291370,2.232209,378.608069,2.038536,0.000031,0.0,0.0,0.0,55.0
1990-04-01,91.45,426.370759,2.332332,378.608069,2.038536,0.000040,0.0,0.0,0.0,55.0
1990-05-01,92.00,436.584900,2.432455,382.464078,2.038536,0.000044,0.0,0.0,0.0,55.0


## Monthly Averages Table

In [44]:
monthly_avg=all_monthly.groupby(all_monthly.index.year).mean()
monthly_avg.head()

,KC Future,Colombia New FX,Honduras New FX,Uganda New FX,Ethiopia New FX,Brazil New FX,Brazilian Rates,Colombian Rates,Honduran Rates,Ugandan Rates
1990,89.608333,451.200914,2.582639,428.411514,2.038536,0.000071,0.0,0.0,0.0,52.500000
1991,85.083333,568.691232,3.784113,733.250947,2.038536,0.000350,0.0,0.0,0.0,45.333333
1992,64.929167,682.642797,4.985587,1132.661649,2.759902,0.001787,0.0,0.0,0.0,47.666667
1993,68.183333,786.238255,6.413333,1191.944046,4.928275,0.032091,0.0,0.0,0.0,27.000000
1994,139.891667,826.351305,8.211938,942.316526,5.228127,0.638295,0.0,0.0,0.0,19.958333


In [45]:
brazil_monthly = monthly_avg[["KC Future", "Brazil New FX", "Brazilian Rates"]]
brazil_monthly.index = index_years
colombia_monthly = monthly_avg[["KC Future", "Colombia New FX", "Colombian Rates"]]
colombia_monthly.index = index_years
honduras_monthly = monthly_avg[["KC Future", "Honduras New FX", "Honduran Rates"]]
honduras_monthly.index = index_years
uganda_monthly = monthly_avg[["KC Future", "Uganda New FX", "Ugandan Rates"]]
uganda_monthly.index = index_years
ethiopia_monthly = monthly_avg[["KC Future", "Ethiopia New FX"]]
ethiopia_monthly.index = index_years

## KC Realized Vol Calculation

In [46]:
# Creating the Log Return Column
coffee_futures_data["LogReturn"] = (np.log(coffee_futures_data.Settle / coffee_futures_data.Settle.shift(-1)))

year_end = ["1990-12-31", "1991-12-31","1992-12-31","1993-12-31","1994-12-31","1995-12-31","1996-12-31","1997-12-31",
            "1998-12-31","1999-12-31","2000-12-31","2001-12-31","2002-12-31","2003-12-31","2004-12-31","2005-12-31",
            "2006-12-31","2007-12-31","2008-12-31","2009-12-31","2010-12-31","2011-12-31","2012-12-31","2013-12-31",
            "2014-12-31","2015-12-31","2016-12-31","2017-12-31","2018-12-31"]

year_start = ["1990-01-02","1991-01-02","1992-01-02","1993-01-02","1994-01-02","1995-01-02","1996-01-02","1997-01-02",
              "1998-01-02","1999-01-02","2000-01-02","2001-01-02","2002-01-02","2003-01-02","2004-01-02","2005-01-02",
              "2006-01-02","2007-01-02","2008-01-02","2009-01-02","2010-01-02","2011-01-02","2012-01-02","2013-01-02",
              "2014-01-02","2015-01-02","2016-01-02","2017-01-02","2018-01-02"]

# Zipping start and end together
combined_years = zip(year_end, year_start)

# Initializing empty lists and dataframes
rvol_by_year = pd.DataFrame()
rvol = []
date_year = []

# Looping through the years in combined years to create a realized vol by year dataframe
for year in combined_years:
        year_data = coffee_futures_data.loc[str(year[0]):str(year[1])]
        daily_std = np.std(year_data.LogReturn)    
        std = float(round((daily_std * 252 **.5), 4) * 100)
        rvol.append(std)

rvol_by_year["Realized Vol"] = rvol
rvol_by_year.index = index_years
kc_rvol = rvol_by_year

## Function for Calculating FX Realized Vol

In [47]:
# Defining function to calculate realized volatility by country
def real_vol(column):
    
    new_fx_data = pd.DataFrame()
    new_fx_data=fx_data[[f"{column}"]].astype(float)
    new_fx_data.index.name = "Date"
    new_fx_data.index = pd.to_datetime(new_fx_data.index)
    new_fx_data.index = new_fx_data.index.strftime('%m-%Y')
    
    new_fx_data["LogReturn"] = (np.log(new_fx_data[f"{column}"] / new_fx_data[f"{column}"].shift(1)))
    
    year_end = ["12-1990", "12-1991","12-1992","12-1993","12-1994","12-1995","12-1996","12-1997","12-1998","12-1999",
                "12-2000","12-2001","12-2002","12-2003","12-2004","12-2005","12-2006","12-2007","12-2008","12-2009","12-2010",
                "12-2011","12-2012","12-2013","12-2014","12-2015","12-2016","12-2017","12-2018"]

    year_start = ["01-1990", "01-1991","01-1992","01-1993","01-1994","01-1995","01-1996","01-1997","01-1998","01-1999",
                  "01-2000","01-2001","01-2002","01-2003","01-2004","01-2005","01-2006","01-2007","01-2008","01-2009","01-2010",
                  "01-2011","01-2012","01-2013","01-2014","01-2015","01-2016","01-2017","01-2018"]
    
    combined_years = zip(year_end, year_start)
    rvol_by_year = pd.DataFrame()
    rvol = []
    date_year = []

    for year in combined_years:
            year_data = new_fx_data.loc[str(year[1]):str(year[0])]
            monthly_std = np.std(year_data.LogReturn)    
            std = float(round((monthly_std * 12 **.5), 4) * 100)
            rvol.append(std)

    rvol_by_year["FX Realized Vol"] = rvol
    rvol_by_year.index = index_years
    
    return rvol_by_year                 

In [48]:
#Setting function to specific country variables
peso_rvol = real_vol("Colombia New FX")
brl_rvol = real_vol("Brazil New FX")
birr_rvol = real_vol("Ethiopia New FX")
lempira_rvol = real_vol("Honduras New FX")
shilling_rvol = real_vol("Uganda New FX")

In [49]:
prices_to_growers.hvplot(title='prices to growers',rot=90)

:NdOverlay   [Variable]
   :Curve   [index]   (value)

##  Creating Master Data Tables

In [50]:
brazil_master_data = pd.concat([brazil_ico, brazil_monthly, brl_rvol, kc_rvol], axis="columns", join="inner")
brazil_master_data = brazil_master_data.replace(',','',regex=True)
brazil_master_data = brazil_master_data.astype(np.float64)
brazil_master_data["Country"] = "Brazil"
brazil_master_data = brazil_master_data.rename(columns={"Brazil New FX":"Currency"})
brazil_master_data = brazil_master_data.rename(columns={"Brazilian Rates":"Rates"})
brazil_master_data["Currency_normalized"]=brazil_master_data["Currency"]*(brazil_master_data["Exports"].mean() / brazil_master_data["Currency"].mean())
brazil_master_data

colombia_master_data = pd.concat([colombia_ico, colombia_monthly, peso_rvol, kc_rvol], axis="columns", join="inner")
colombia_master_data = colombia_master_data.replace(',','',regex=True)
colombia_master_data = colombia_master_data.astype(np.float64)
colombia_master_data["Country"] = "Colombia"
colombia_master_data = colombia_master_data.rename(columns={"Colombia New FX":"Currency"})
colombia_master_data = colombia_master_data.rename(columns={"Colombian Rates":"Rates"})
colombia_master_data["Currency_normalized"]=colombia_master_data["Currency"]*(colombia_master_data["Exports"].mean() / colombia_master_data["Currency"].mean())


honduras_master_data = pd.concat([honduras_ico, honduras_monthly, lempira_rvol, kc_rvol], axis="columns", join="inner")
honduras_master_data = honduras_master_data.replace(',','',regex=True)
honduras_master_data = honduras_master_data.astype(np.float64)
honduras_master_data["Country"] = "Honduras"
honduras_master_data = honduras_master_data.rename(columns={"Honduras New FX":"Currency"})
honduras_master_data = honduras_master_data.rename(columns={"Honduran Rates":"Rates"})
honduras_master_data["Currency_normalized"]=honduras_master_data["Currency"]*(honduras_master_data["Exports"].mean() / honduras_master_data["Currency"].mean())

uganda_master_data = pd.concat([uganda_ico, uganda_monthly, shilling_rvol, kc_rvol], axis="columns", join="inner")
uganda_master_data = uganda_master_data.replace(',','',regex=True)
uganda_master_data = uganda_master_data.astype(np.float64)
uganda_master_data["Country"] = "Uganda"
uganda_master_data = uganda_master_data.rename(columns={"Uganda New FX":"Currency"})
uganda_master_data = uganda_master_data.rename(columns={"Ugandan Rates":"Rates"})
uganda_master_data["Currency_normalized"]=uganda_master_data["Currency"]*(uganda_master_data["Exports"].mean() / uganda_master_data["Currency"].mean())

ethiopia_master_data = pd.concat([ethiopia_ico, ethiopia_monthly, birr_rvol, kc_rvol], axis="columns", join="inner")
ethiopia_master_data = ethiopia_master_data.replace(',','',regex=True)
ethiopia_master_data = ethiopia_master_data.astype(np.float64)
ethiopia_master_data["Country"] = "Ethiopia"
ethiopia_master_data = ethiopia_master_data.rename(columns={"Ethiopia New FX":"Currency"})
ethiopia_master_data = ethiopia_master_data.rename(columns={"Ethiopian Rates":"Rates"})
ethiopia_master_data["Currency_normalized"]=ethiopia_master_data["Currency"]*(ethiopia_master_data["Exports"].mean() / ethiopia_master_data["Currency"].mean())

# Creating master all list
all_master_data = pd.concat([brazil_master_data, colombia_master_data, honduras_master_data, uganda_master_data, ethiopia_master_data], axis='rows', join='inner')

In [51]:
country_locations = Path('resources/country_locations.csv')
locations = pd.read_csv(coffee_futures, parse_dates=True, index_col='Date')

In [52]:
print(brazil_master_data.columns)
print(colombia_master_data.columns)
print(ethiopia_master_data.columns)
print(honduras_master_data.columns)
print(uganda_master_data.columns)

Index(['Production', 'Exports', 'Prices to Growers', 'Opening Stocks',
       'KC Future', 'Currency', 'Rates', 'FX Realized Vol', 'Realized Vol',
       'Country', 'Currency_normalized'],
      dtype='object')
Index(['Production', 'Exports', 'Prices to Growers', 'Opening Stocks',
       'KC Future', 'Currency', 'Rates', 'FX Realized Vol', 'Realized Vol',
       'Country', 'Currency_normalized'],
      dtype='object')
Index(['Production', 'Exports', 'Prices to Growers', 'Opening Stocks',
       'KC Future', 'Currency', 'FX Realized Vol', 'Realized Vol', 'Country',
       'Currency_normalized'],
      dtype='object')
Index(['Production', 'Exports', 'Prices to Growers', 'Opening Stocks',
       'KC Future', 'Currency', 'Rates', 'FX Realized Vol', 'Realized Vol',
       'Country', 'Currency_normalized'],
      dtype='object')
Index(['Production', 'Exports', 'Prices to Growers', 'Opening Stocks',
       'KC Future', 'Currency', 'Rates', 'FX Realized Vol', 'Realized Vol',
       'Country', 

In [53]:
all_master_data["%chgProd"] = all_master_data["Production"].pct_change()*100
all_master_data["%chgExports"] = all_master_data["Exports"].pct_change()*100
all_master_data["%chgPTG"] = all_master_data["Prices to Growers"].pct_change()*100
all_master_data["differential"] = all_master_data["KC Future"] - all_master_data["Prices to Growers"]



In [86]:
all_master_data_pctchg = pd.DataFrame()
all_master_data_pctchg["pctProd"] = all_master_data["Production"].pct_change()*100 
all_master_data_pctchg["pctExports"] = all_master_data["Exports"].pct_change()*100
all_master_data_pctchg["pctPTG"] = all_master_data["Prices to Growers"].pct_change()*100
all_master_data_pctchg["pctOS"] = all_master_data["Opening Stocks"].pct_change()*100
all_master_data_pctchg["pctKC"] = all_master_data["KC Future"].pct_change()*100
all_master_data_pctchg["pctCurrency"] = all_master_data["Currency"].pct_change()*100
all_master_data_pctchg["FXRVol"] = all_master_data["FX Realized Vol"]
all_master_data_pctchg["KCRVol"] = all_master_data["Realized Vol"]
all_master_data_pctchg["Country"] = all_master_data["Country"]


In [87]:
all_master_data_pctchg

,pctProd,pctExports,pctPTG,pctOS,pctKC,pctCurrency,FXRVol,KCRVol,Country
0,NaN,NaN,NaN,NaN,NaN,NaN,90.90,28.47,Brazil
1,0.025654,22.084756,-19.016937,-5.117730,-5.049754,392.633805,75.33,23.98,Brazil
2,26.783424,-23.184153,2.750625,-11.441209,-23.687561,410.656700,52.21,33.01,Brazil
3,-18.599543,11.992598,16.991150,27.827624,5.011872,1695.523338,10.26,42.53,Brazil
4,0.088756,-14.940568,116.660363,1.058055,105.169885,1888.993021,61.45,59.47,Brazil
5,-35.939274,-13.773656,0.829187,10.038663,5.468517,43.642727,5.63,36.37,Brazil
6,61.666667,25.436047,-16.594529,-18.034265,-21.971195,9.583715,0.32,40.38,Brazil
7,-10.442854,-11.054461,50.918526,3.554493,60.340210,7.259984,0.33,60.92,Brazil
8,40.588190,37.343669,-27.618458,-2.578944,-27.599205,7.653670,0.50,37.41,Brazil
9,29.425206,0.479014,-29.624703,13.070617,-20.916602,56.384842,33.52,54.58,Brazil


In [54]:
all_master_data["Currency"].mean()

754.821723562578

In [55]:
all_master_data["%chg_differential"] = (all_master_data["differential"] / all_master_data["KC Future"])*100

In [56]:
all_master_data.head()

,Production,Exports,Prices to Growers,Opening Stocks,KC Future,Currency,FX Realized Vol,Realized Vol,Country,Currency_normalized,%chgProd,%chgExports,%chgPTG,differential,%chg_differential
1990,27286.0,17863.0,54.32,26926.0,89.608333,0.000071,90.90,28.47,Brazil,0.994263,NaN,NaN,NaN,35.288333,39.380638
1991,27293.0,21808.0,43.99,25548.0,85.083333,0.000350,75.33,23.98,Brazil,4.898075,0.025654,22.084756,-19.016937,41.093333,48.297747
1992,34603.0,16752.0,45.20,22625.0,64.929167,0.001787,52.21,33.01,Brazil,25.012348,26.783424,-23.184153,2.750625,19.729167,30.385677
1993,28167.0,18761.0,52.88,28921.0,68.183333,0.032091,10.26,42.53,Brazil,449.102549,-18.599543,11.992598,16.991150,15.303333,22.444390
1994,28192.0,15958.0,114.57,29227.0,139.891667,0.638295,61.45,59.47,Brazil,8932.618349,0.088756,-14.940568,116.660363,25.321667,18.100911


In [57]:
all_master_data.hvplot.line(
x="index",
y=["Currency_normalized","Exports"], 
    xlabel="FX RVOL", 
    ylabel="YEAR",
    rot = 90,
    shared_axes=False,
    groupby="Country").opts(framewise=False)


:DynamicMap   [Country]
   :NdOverlay   [Variable]
      :Curve   [index]   (value)

In [58]:
all_master_data = all_master_data.reset_index()

In [59]:
country_year_grp = all_master_data[["%chgProd","%chgExports","%chgPTG","Country"]].groupby('Country')

In [60]:
all_master_data.to_csv("all_master_data.csv")

In [61]:
all_master_data.hvplot.line(
x="index",
y=["FX Realized Vol","%chgExports"], 
    xlabel="FX RVOL", 
    ylabel="YEAR",
    rot = 90,
    shared_axes=False,
    groupby="Country")


:DynamicMap   [Country]
   :NdOverlay   [Variable]
      :Curve   [index]   (value)

In [62]:
production_exports_bar = all_master_data.hvplot.bar(
x="index",
y=["Production","Exports"],
    stacked=True,
    xlabel="X", 
    ylabel="UNIT",
    rot = 90,
    groupby="Country")

production_exports_bar

:DynamicMap   [Country]
   :Bars   [index,Variable]   (value)

In [63]:
all_master_data.hvplot.line(
x="index",
y=["FX Realized Vol","%chgExports"], 
    xlabel="FX RVOL", 
    ylabel="YEAR",
    rot = 90,
    shared_axes=False,
    groupby="Country")

:DynamicMap   [Country]
   :NdOverlay   [Variable]
      :Curve   [index]   (value)

In [64]:
realized_vol_line = all_master_data.iloc[0:28].loc[all_master_data["Country"]=="Brazil"].hvplot.line(
x="index",
y=["Realized Vol"],
xlabel= "X2",
ylabel="UNITs",
rot=90,
)

realized_vol_line

:Curve   [index]   (Realized Vol)

In [65]:
all_master_data.loc[0:28]

,index,Production,Exports,Prices to Growers,Opening Stocks,KC Future,Currency,FX Realized Vol,Realized Vol,Country,Currency_normalized,%chgProd,%chgExports,%chgPTG,differential,%chg_differential
0,1990,27286.0,17863.0,54.32,26926.0,89.608333,0.000071,90.90,28.47,Brazil,0.994263,NaN,NaN,NaN,35.288333,39.380638
1,1991,27293.0,21808.0,43.99,25548.0,85.083333,0.000350,75.33,23.98,Brazil,4.898075,0.025654,22.084756,-19.016937,41.093333,48.297747
2,1992,34603.0,16752.0,45.20,22625.0,64.929167,0.001787,52.21,33.01,Brazil,25.012348,26.783424,-23.184153,2.750625,19.729167,30.385677
3,1993,28167.0,18761.0,52.88,28921.0,68.183333,0.032091,10.26,42.53,Brazil,449.102549,-18.599543,11.992598,16.991150,15.303333,22.444390
4,1994,28192.0,15958.0,114.57,29227.0,139.891667,0.638295,61.45,59.47,Brazil,8932.618349,0.088756,-14.940568,116.660363,25.321667,18.100911
5,1995,18060.0,13760.0,115.52,32161.0,147.541667,0.916864,5.63,36.37,Brazil,12831.056594,-35.939274,-13.773656,0.829187,32.021667,21.703474
6,1996,29197.0,17260.0,96.35,26361.0,115.125000,1.004734,0.32,40.38,Brazil,14060.748453,61.666667,25.436047,-16.594529,18.775000,16.308360
7,1997,26148.0,15352.0,145.41,27298.0,184.591667,1.077677,0.33,60.92,Brazil,15081.556497,-10.442854,-11.054461,50.918526,39.181667,21.226130
8,1998,36761.0,21085.0,105.25,26594.0,133.645833,1.160159,0.50,37.41,Brazil,16235.849037,40.588190,37.343669,-27.618458,28.395833,21.247077
9,1999,47578.0,21186.0,74.07,30070.0,105.691667,1.814313,33.52,54.58,Brazil,25390.406918,29.425206,0.479014,-29.624703,31.621667,29.918789


In [66]:
all_master_data.hvplot.line(
x="index",
y=["KC Future","Prices to Growers", "Realized Vol"], 
    xlabel="differential", 
    ylabel="UNIT",
    rot = 90,
    groupby="Country")

:DynamicMap   [Country]
   :NdOverlay   [Variable]
      :Curve   [index]   (value)

## Correlation Tables for Visualization

In [97]:
brazil_corr = all_master_data_pctchg.loc[all_master_data_pctchg["Country"]=="Brazil"].corr()
colombia_corr = all_master_data_pctchg.loc[all_master_data_pctchg["Country"]=="Colombia"].corr()
honduras_corr = all_master_data_pctchg.loc[all_master_data_pctchg["Country"]=="Honduras"].corr()
uganda_corr = all_master_data_pctchg.loc[all_master_data_pctchg["Country"]=="Uganda"].corr()
ethiopia_corr = all_master_data_pctchg.loc[all_master_data_pctchg["Country"]=="Ethiopia"].corr()

master_corr = pd.concat([brazil_corr, colombia_corr, honduras_corr, uganda_corr, ethiopia_corr],axis=0,join="inner")

In [98]:
master_corr

,pctProd,pctExports,pctPTG,pctOS,pctKC,pctCurrency,FXRVol,KCRVol
pctProd,1.000000,0.561901,-0.280548,-0.618099,-0.248923,-0.138512,0.049766,-0.011496
pctExports,0.561901,1.000000,-0.452703,-0.152814,-0.354587,-0.123582,-0.130267,-0.153946
pctPTG,-0.280548,-0.452703,1.000000,-0.008245,0.965300,0.442571,0.143015,0.373444
pctOS,-0.618099,-0.152814,-0.008245,1.000000,-0.015565,0.134421,-0.078252,0.092423
pctKC,-0.248923,-0.354587,0.965300,-0.015565,1.000000,0.394356,0.141828,0.426225
pctCurrency,-0.138512,-0.123582,0.442571,0.134421,0.394356,1.000000,0.492500,0.387069
FXRVol,0.049766,-0.130267,0.143015,-0.078252,0.141828,0.492500,1.000000,-0.038616
KCRVol,-0.011496,-0.153946,0.373444,0.092423,0.426225,0.387069,-0.038616,1.000000
pctProd,1.000000,0.842992,0.157716,0.299667,0.139204,-0.723005,0.025229,0.095840
pctExports,0.842992,1.000000,-0.097759,0.660140,-0.161154,-0.700723,-0.016718,-0.173612


In [89]:
all_master_data_pctchg.corr

px.imshow(all_master_data_pctchg)



<bound method DataFrame.corr of        pctProd  pctExports      pctPTG       pctOS       pctKC   pctCurrency  \
0          NaN         NaN         NaN         NaN         NaN           NaN   
1     0.025654   22.084756  -19.016937   -5.117730   -5.049754    392.633805   
2    26.783424  -23.184153    2.750625  -11.441209  -23.687561    410.656700   
3   -18.599543   11.992598   16.991150   27.827624    5.011872   1695.523338   
4     0.088756  -14.940568  116.660363    1.058055  105.169885   1888.993021   
5   -35.939274  -13.773656    0.829187   10.038663    5.468517     43.642727   
6    61.666667   25.436047  -16.594529  -18.034265  -21.971195      9.583715   
7   -10.442854  -11.054461   50.918526    3.554493   60.340210      7.259984   
8    40.588190   37.343669  -27.618458   -2.578944  -27.599205      7.653670   
9    29.425206    0.479014  -29.624703   13.070617  -20.916602     56.384842   
10  -34.192274  -12.314736  -10.922101   45.533755  -13.813766      0.862890   
11    0.

***

***

# PCA Analysis

In [33]:
df = brazil_master_data
features = ['Production', 'Exports', 'Opening Stocks','KC Future',
            'Brazil New FX', 'Brazilian Rates', 'FX Realized Vol',
            'Realized Vol']
x = df.loc[:,features].values
y = df.loc[:,['Prices to Growers']].values

x = StandardScaler().fit_transform(x)

pca = PCA()
principalComponents = pca.fit_transform(x)
principal_df = pd.DataFrame(data = principalComponents, 
                            columns = ['Production', 'Exports', 'Opening Stocks',
                                       'KC Future', 'Brazil New FX', 'Brazilian Rates', 'FX Realized Vol',
                                       'Realized Vol'])
target_df = pd.DataFrame(data=y,
                        columns=["Prices to Growers"]
                        )
final_df = pd.concat([principal_df, target_df], axis = 1)

C:\Users\Daniel Singer\anaconda3\envs\pyvizenv\lib\site-packages\pandas\core\indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)
C:\Users\Daniel Singer\anaconda3\envs\pyvizenv\lib\site-packages\sklearn\utils\extmath.py:770: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\Daniel Singer\anaconda3\envs\pyvizenv\lib\site-packages\sklearn\utils\extmath.py:711: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = op(x, *args, **kwargs)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
labels = {
    str(i): f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}

fig = px.scatter_matrix(
    principalComponents,
    labels=labels,
    dimensions=range(8),
    color=df["Prices to Growers"],
    height=1100
)
fig.update_traces(diagonal_visible=False)
fig.show()

In [ ]:
pca = PCA()
pca.fit(principalComponents)
exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)

px.area(
    x=range(1, exp_var_cumul.shape[0] + 1),
    y=exp_var_cumul,
    labels={"x": "# Components", "y": "Explained Variance"}
)